# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: Thought: To analyze the latest news and updates related to the financial markets, particularly Inflation in the US and its impact on the stock market in 2024, I need to find relevant online news sources or websites that provide real-time updates on these topics.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest news on inflat



# Agent: Lead Market Analyst
## Thought: Thought: Based on the information found on the CNBC website, it appears that the personal consumption expenditures price index (PCE) has increased by 0.2% on the month and showed a 12-month inflation rate of 2.3%, meeting expectations.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.reuters.com/markets/us/futures-dip-caution-prevails-ahead-inflation-data-2024-11-27/\"}"
## Tool Output: 
Nasdaq leads Wall St declines as tech stocks slump after inflation data | Reuters
Skip to main contentExclusive news, data and analytics for financial market professionalsLearn more aboutRefinitivWorldBrowse WorldAfricaAmericasAsia PacificChinaEuropeIndiaIsrael and Hamas at WarJapanMiddle EastUkraine and Russia at WarUnited KingdomUnited StatesReuters NEXTUS ElectionBusinessBrowse BusinessAerospace & DefenseAutos & TransportationDavosEnergyEnvironmentFinanceHealthcare & PharmaceuticalsMedia & TelecomRetail & ConsumerFuture o



# Agent: Lead Market Analyst
## Thought: Thought: I've gathered information on the latest news and updates related to Inflation in the US and its impact on the stock market in 2024 from various sources, including CNBC and Reuters. Now, I need to analyze this information and identify the most relevant and impactful news items that could influence market trends or investor decisions.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"impact of inflation on stock market 2024\"}"
## Tool Output: 

Search results: Title: Inflation Artificially Pumps Up the Stock Market
Link: https://www.heritage.org/budget-and-spending/commentary/inflation-artificially-pumps-the-stock-market
Snippet: Inflation is also making stock markets appear stronger than they really are and cutting into returns for everyone, including those with retirement accounts.
---
Title: How Inflation Affects The Stock Market | Bankrate
Link: https://www.bankrate.com/investing/how-inflation-affects-the-sto



# Agent: Lead Market Analyst
## Thought: Thought: Now that I've gathered a wide range of information on inflation in the US in 2024 and its effects on the economy, I'll analyze this information to identify key trends and insights that could influence market trends or investor decisions.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.morningstar.com/economy/why-we-expect-inflation-fall-2024\"}"
## Tool Output: 
Why We Expect Inflation to Fall in 2024 | Morningstar
					Morningstar brands and products
					Company
		Portfolio
		Tools Sections Markets 
		Funds
		ETFs
		Stocks
		Bonds
		 Investing Ideas 
		Home
 Tools 
		Portfolio
		Watchlists
		Screener
		Chart
		Rating Changes
 Sections Markets 
		Funds
		ETFs
		Stocks
		Bonds
 Investing Ideas 
		Help
		What’s New
 Products for Investors 
		All Products and Services
 Table of Contents 
						Summary
						How Did Inflation Fall Without a Recession?
						What Is the Current Inflation Rate in the Un



# Agent: Lead Market Analyst
## Final Answer: 
Inflation in the US in 2024 is expected to continue its downward trend, with core PCE inflation projected to fall to 2.0% by the end of the year. This decline is driven by improvements in supply chain conditions, a moderation in wage growth, and a decrease in housing inflation. Despite these positive developments, there are still risks to the inflation outlook, including potential disruptions to the global supply chain and the impact of the 2024 presidential election on the economy.

The Federal Reserve has taken steps to address inflation, including interest rate hikes, and these efforts are expected to continue to weigh on inflation in the coming year. As a result, the Fed's preferred inflation gauge, the Personal Consumption Expenditures Price Index (PCE), is expected to fall to around 2.3% by the end of 2024.

Investors should be aware that the decline in inflation will likely have differing impacts on various market sectors. Stocks 

...



# Agent: Chief Data Strategist
## Thought: Thought: I now know some key trends and risks related to Inflation in the US and the impact on the stock market in 2024 from a reputable news source. However, I need to gather more information about the overall impact of inflation on various market sectors and investor sentiment.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"impact of inflation on stock market sectors and investor sentiment in 2024\", \"website\": \"https://www.cnbc.com/\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the stor



# Agent: Chief Data Strategist
## Thought: Thought: I have gathered relevant information about the impact of inflation on market sectors and investor sentiment in 2024 from various sources.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"inflation impact on market sectors and investor sentiment 2024 trends\"}"
## Tool Output: 

Search results: Title: Analysis: Assessing Inflation's Impact - U.S. Bank
Link: https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html
Snippet: While over the course of 2024 inflation has trended lower, yields on 10-year U.S. Treasury bonds recently moved higher. After the Fed's initial ...
---
Title: How Inflation Affects The Stock Market | Bankrate
Link: https://www.bankrate.com/investing/how-inflation-affects-the-stock-market/
Snippet: 1. Inflation influences stock prices. High inflation can affect stock prices in different ways depending on the strength of the business.

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00, 25.85it/s]




# Agent: Creative Content Director
## Thought: Thought: I've gathered some specific insights related to US inflation trends and their impact on the stock market in 2024. Now, I can create high-quality content pieces that will educate and inform my audience.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation trends 2024 impact on stock market\", \"website\": \"https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html\"}"
## Tool Output: 
Relevant Content:
investment strategy director for U.S. Bank Asset Management. Fed Chair Jerome Powell in early November stated that “the election will have no effects on our policy decisions.” Powell added, “In considering additional adjustments to the target range for the federal funds rate, the Committee will carefully assess incoming data, the evolving outlook and the balance of risks. We are not on a preset course.”7 Inflation’s recent history 



# Agent: Chief Content Officer
## Final Answer: 
```markdown
# Understanding US Inflation Trends and their Impact on the Stock Market in 2024

## Introduction

As we head into 2024, the US economy is experiencing a notable shift in inflation trends and their effects on the stock market. The core Personal Consumption Expenditures (PCE) inflation rate is projected to decline to **2.0%** by the end of the year, which could signal a more stable economic environment for investors. This blog post will explore the implications of these trends and how investors can position themselves for success in 2024.

In the context of ongoing macroeconomic changes, understanding the dynamics of inflation and its relation to market performance is essential. As inflation affects consumer spending and business investment, investors need to adapt their strategies to leverage these trends effectively.

## Inflation Trends: The Current Landscape

According to recent reports, inflation in the US has shown sig

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
🚨 **Inflation Update for 2024!** Core PCE
inflation is projected to fall to 2.0% by the
year’s end, promising a more stable economic
outlook. How will this impact your investment
strategy? #USInflation #StockMarketTrends
--------------------------------------------------
Twitter
📈 **S&P 500 Gains!** Anticipated to grow over 20%
in 2024! As inflation declines, equities may hold
the key to unlocking potential market returns.
Don’t miss out! #InvestSmart #Markets
--------------------------------------------------
Twitter
⚖️ **Investing Strategy Alert:** As inflation
trends down, consider increasing your equity
allocation and maintaining balance in real assets.
Knowledge is power! #InvestmentTips
#WealthManagement
--------------------------------------------------
Twitter
💬 **Join the Conversation:** What are your
thoughts on the impact of inflation and interest
rates on the stock market this year? Share your
insights below! #FinanceCommunity #StockTalk
----------------------------

## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Understanding US Inflation Trends and their Impact on the Stock Market in 2024

## Introduction

As we head into 2024, the US economy is experiencing a notable shift in inflation trends and their effects on the stock market. The core Personal Consumption Expenditures (PCE) inflation rate is projected to decline to **2.0%** by the end of the year, which could signal a more stable economic environment for investors. This blog post will explore the implications of these trends and how investors can position themselves for success in 2024.

In the context of ongoing macroeconomic changes, understanding the dynamics of inflation and its relation to market performance is essential. As inflation affects consumer spending and business investment, investors need to adapt their strategies to leverage these trends effectively.

## Inflation Trends: The Current Landscape

According to recent reports, inflation in the US has shown signs of cooling down. In **October 2024**, the annual inflation rate stood at **2.6%**, a decrease from higher levels seen in previous years. Improvements in supply chain conditions and stable housing costs have been contributing factors to this decline. However, risks such as potential global supply chain disruptions and the ongoing political climate pose challenges to the inflation outlook.

The shift in inflation rates indicates a broader trend towards stabilization, influenced by several key variables, including consumer behavior and production costs. Investors should closely monitor these factors to assess how different sectors might respond to changing inflation dynamics.

![US Inflation Trends Chart](https://example.com/inflation-trends-chart)  
*Figure 1: Projected US Inflation Rate 2024-2026*

## The Federal Reserve's Role

The Federal Reserve has actively engaged in managing inflation through interest rate adjustments. For instance, recent interest rate hikes are expected to play a critical role in guiding inflation lower. As Fed Chair Jerome Powell and other policymakers evaluate incoming economic data, their decisions will have lasting implications for both inflation and investor sentiment in 2024.

Understanding the Federal Reserve’s approach to monetary policy will be pivotal for investors. As the Fed seeks to balance economic growth with inflation control, its strategies will ripple through various sectors and impact market dynamics significantly.

![Fed Interest Rate Projections](https://example.com/fed-interest-rate-chart)  
*Figure 2: Fed Interest Rate Projections for 2024*

## Impact on the Stock Market

The decline in inflation is expected to have mixed impacts on various sectors of the stock market. Historically, lower inflation can enhance investor confidence and lead to increased equity market performance. The S&P 500 is anticipated to gain over **20%** again in 2024. However, potential interest rate hikes could offset these benefits as borrowing costs rise.

In practice, certain sectors such as technology and consumer discretionary may experience a boost, while others like utilities and financials might face challenges depending on how they adapt to the changes in inflation and interest rates. It’s crucial for investors to identify these performance trends to optimize their portfolios strategically.

![Stock Market Performance Chart](https://example.com/stock-market-performance-chart)  
*Figure 3: S&P 500 Performance in 2024*

## Recommendations for Investors

As inflation continues to trend downward, investors should consider the following strategies:

1. **The Case for Equities:** With the S&P 500 on a growth trajectory, allocating more toward equities may yield better returns than fixed income investments. Maintaining a balanced approach is essential. Diversifying across sectors could help mitigate risks associated with volatile market conditions.

2. **Diversity in Real Assets:** Keep a neutral allocation in real assets to hedge against any unexpected inflationary pressures. Assets such as real estate and commodities can provide a buffer during uncertain times, as they often perform well when traditional equity markets fluctuate.

3. **Stay Informed:** Continuous monitoring of economic indicators and Federal Reserve policy updates will be crucial for making informed decisions in a changing landscape. Regularly update your investment thesis based on new data to ensure your strategies remain relevant and effective.

## Conclusion

Though the inflation outlook appears to be improving heading into 2024, investors must remain vigilant. Navigating the complexities of the market amid ongoing risks will require astute analysis and adaptability. By staying informed and adjusting their investment strategies accordingly, investors can better position themselves for potential market success in the coming year.

Investors should not only look at historical trends but also consider future projections and market fundamentals as part of their decision-making process. Proactive engagement with economic developments will enhance the likelihood of favorable investment outcomes.